# [Collision Avoidance Challenge](https://kelvins.esa.int/collision-avoidance-challenge/home/)
## Introduction
Today, active **collision avoidance** among orbiting satellites has become a routine task in space operations, relying on validated, accurate and timely space surveillance data. For a typical satellite in Low Earth Orbit, hundreds of alerts are issued every week corresponding to possible close encounters between a satellite and another space object (in the form of conjunction data messages CDMs). After automatic processing and filtering, there remain about 2 actionable alerts per spacecraft and week, requiring detailed follow-up by an analyst. On average, at the European Space Agency, more than one collision avoidance manoeuvre is performed per satellite and year.


In this challenge, you are tasked to build a model to predict the final collision risk estimate between a given satellite and a space object (e.g. another satellite, space debris, etc). To do so, you will have access to a database of real-world conjunction data messages (CDMs) carefully prepared at ESA. Learn more about the challenge and the data.

This competition is organized by ESA's Advanced Concepts Team (ACT) in partnership with ESA's Space Debris Office

Experts from both teams are available for interactions during the competition.

## Challenge

As of estimations done in January 2019, more than 34,000 objects with a size larger than 10cm are orbiting our planet. Of these, 22,300 are tracked by the Space Surveillance Network and their position released in the form of a globally shared catalogue.


ESA's Space Debris Office supports collision avoidance activities covering the ESA missions Aeolus, Cryosat-2 and the constellation of Swarm-A/B/C in low-Earth orbit and Cluster-II in highly eccentric orbit approaching the Geostationary (GEO) region. On top of these, more than a dozen spacecraft of partner agencies and commercial operators are supported.

![Alt text](https://kelvins.esa.int/media/public/ckeditor_uploads/2021/08/05/new_swarm.png "Title")

In the context of this activity, the orbits of these satellites are propagated and when a close approach with any object in the catalogue is detected a **Conjunction Data Message** (CDM) is assembled and released. Each CDM **contains multiple attributes** about the approach, such as the identity of the satellite in question, the object type of the potential collider, the time of closest approach (TCA), the uncertainty (i.e. covariances), etc. It also contains a **self-reported** risk, which is computed using some of the attributes from the CDM. In the days following the first CDM, as the uncertainties of the objects positions become smaller, other CDMs are released refining the knowledge acquired on the close encounter.

Typically, a **time series** of CDMs covering one week is released **for each unique close approach**, with about 3 CDMs becoming available per day. For a given close approach the last obtained CDM, including the computed risk, can be assumed to be the **best knowledge** we have about the potential collision and the state of the two objects in question. In most cases, the Space Debris Office will alarm control teams and start thinking about a potential avoidance manoeuvre 2 days prior to the close approach in order to avoid the risk of collision, to then make a final decision 1 day prior. In this challenge, we ask to **build a model** that makes use of the CDMs recorded up to 2 days prior to the closest approach to **predict the final risk** (i.e. the risk predicted in the last available CDM prior to close approach).

More about the dataset used in this competition and the attributes contained in the various CDMs can be found in the data section. You can also learn some more about the current way ESA's Space Debris office deals with collision avoidance manoeuvres reading this [paper](https://kelvins.esa.int/media/public/competitions/collision-avoidance-challenge/SDC7-paper1017.pdf).

We thank the US Space Surveillance Network for the provision of surveillance data supporting safe operations of ESA’s spacecraft. Specifically, we are grateful to the agreement which allows to publicly release the dataset for the purpose of this competition.

## Data

### Differences Between Training and Testing Data
Each dataset is made of several unique events (close encounters betwen two objects) which are indexed by a unique number in the `event_i` column.

- The `training` set has 162634 rows and **13154 unique events** (giving on average about 12 rows/CDMs per close encounter).

- The `testing` set has 24484 rows and **2167 unique events** (giving on average about 11 rows/CDMs per close encounter).

**Important: Note that the `testing` set and the `training` set have not been randomly sampled from the database. In other words, while they come from the same database, with the same collection process and the same features, they have been hand picked in order to over-represent high risk events and to create an interesting predictive model. This is a characterstic of this competition where high risk events are scarce, but represent the true final target of a useful predictive model.**

In particular, the `testing` data differs in two major ways compared to the `training` set:

 * It only contains events for which the latest CDM is within 1 day ( `time_to_tc` < 1) of the time to closest approach (TCA). This is because, in some cases, the latest available CDM is days away from the (known) time to closest approach. It would be wrong to assume that the computed risk 7 days before the actual time to closest approach can be a good approximation to the risk at TCA. Furthermore, predicting the risk many days prior the time to closest approach is not of great interest to us. On the other hand, the `training` set is unfiltered and you will find many cases where the latest available CDMs is days away from the TCA. We have chosen to keep these collision events in the training set because they may still be useful when it comes to predicting events from the test set.

 * There are no CDMs to learn from which are within 2 days of the TCA. In other words, the data available closest to the TCA will be at least 2 days away. This is because, as mentioned in the challenge section, a potential avoidance manoeuvre is planned at least 2 days prior to closest approach. Similarly to the above, the `training` set will contain all cases, including events where no data is available at least 2 days prior to closest approach (i.e. events with all their CDMs being within 2 days of TCA are still present in the dataset).
 
### Columns Description

The dataset is represented as a table, where each row correspond to a single CDM, and each CDM contains 103 recorded characteristics/features. There are thus 103 columns, which we describe below. The dataset is made of several unique collision/close approach events, which are identified in the `event_id` column. In turn, each collision event is made of several CDMs recorded over time. Therefore, a single collision event can be thought of as a times series of CDMs. From these CDMs, for every collision event, we are interested in predicting the final risk which is computed in the last CDM of the time series (i.e. the risk value in the last row of each collision event).

For the column description, we first describe columns which have unique names and then the columns whose name difference only depends on whether they are referring to the target object (if the column name starts with a **t**) or the chaser object (if the column name starts with a **c**). Here, target refers to the ESA satellites while chaser refers to the space debris/object we want to avoid. describe the column names shared for both the chaser and the target, we replace **t** and **c** with the placeholder **x**. For instance, `c_sigma_r` and `t_sigma_r` both correspond to the description of `x_sigma_r`.

Note that all the columns are numerical except for `c_object_type`.

#### Uniquely Named Columns
- `risk`:self-computed value at the epoch of each CDM [base 10 log]. **In the test set, this value is to be predicted, at the time of closest approach for each `event_id`. Note that, as mentioned above, in the `test` set, we do not know the actual data contained in CDMs that are within 2 days to closest approach, since they happen in the "future".**
- `event_id`: unique id per collision event
- `time_to_tca`: Time interval between CDM creation and time-of-closest approach [days]
- `mission_id`: identifier of mission that will be affected
- `max_risk_estimate`: maximum collision probability obtained by scaling combined covariance
- `max_risk_scaling`: scaling factor used to compute maximum collision probability
- `miss_distance`: relative position between chaser & target at tca [m]
- `relative_speed`: relative speed between chaser & target at tca [m/s]
- `relative_position_n`: relative position between chaser & target: normal (cross-track) [m]
- `relative_position_r`: relative position between chaser & target: radial [m]
- `relative_position_t`: relative position between chaser & target: transverse (along-track) [m]
- `relative_velocity_n`: relative velocity between chaser & target: normal (cross-track) [m/s]
- `relative_velocity_r`: relative velocity between chaser & target: radial [m/s]
- `relative_velocity_t`: relative velocity between chaser & target: transverse (along-track) [m/s]
- `c_object_type`: object type which is at collision risk with satellite
- `geocentric_latitude`: Latitude of conjunction point [deg]
- `azimuth`: relative velocity vector: azimuth angle [deg]
- `elevation`: relative velocity vector: elevation angle [deg]
- `F10`: 10.7 cm radio flux index [10−2210−22 W/(m2m2 Hz)]
- `AP`: daily planetary geomagnetic amplitude index
- `F3M`: 81-day running mean of F10.7 (over 3 solar rotations) [10−2210−22 W/(m2m2 Hz)]
- `SSN`: Wolf sunspot number

#### Shared Column Names Between the Chaser and the Target Object
 - `x_sigma_rdot`: covariance; radial velocity standard deviation (sigma) [m/s]
 - `x_sigma_n`: covariance; (cross-track) position standard deviation (sigma) [m]
 - `x_cn_r`: covariance; correlation of normal (cross-track) position vs radial position
 - `x_cn_t`: covariance; correlation of normal (cross-track) position vs transverse (along-track) position
 - `x_cndot_n`: covariance; correlation of normal (cross-track) velocity vs normal (cross-track) position
 - `x_sigma_ndot`: covariance; normal (cross-track) velocity standard deviation (sigma) [m/s]
 - `x_cndot_r`: covariance; correlation of normal (cross-track) velocity vs radial position
 - `x_cndot_rdot`: covariance; correlation of normal (cross-track) velocity vs radial velocity
 - `x_cndot_t`: covariance; correlation of normal (cross-track) velocity vs transverse (along-track) position
 - `x_cndot_tdot`: covariance; correlation of normal (cross-track) velocity vs transverse (along-track) velocity
 - `x_sigma_r`: covariance; radial position standard deviation (sigma) [m]
 - `x_ct_r`: covariance; correlation of transverse (along-track) position vs radial position
 - `x_sigma_t`: covariance; transverse (along-track) position standard deviation (sigma) [m]
 - `x_ctdot_n`: covariance; correlation of transverse (along-track) velocity vs normal (cross-track) position
 - `x_crdot_n`: covariance; correlation of radial velocity vs normal (cross-track) position
 - `x_crdot_t`: covariance; correlation of radial velocity vs transverse (along-track) position
 - `x_crdot_r`: covariance; correlation of radial velocity vs radial position
 - `x_ctdot_r`: covariance; correlation of transverse (along-track) velocity vs radial position
 - `x_ctdot_rdot`: covariance; correlation of transverse (along-track) velocity vs radial velocity
 - `x_ctdot_t`: covariance; correlation of transverse (along-track) velocity vs transverse (along-track) position
 - `x_sigma_tdot`: covariance; transverse (along-track) velocity standard deviation (sigma) [m/s]
 - `x_position_covariance_det`: determinant of covariance (~volume)
 - `x_cd_area_over_mass`: ballistic coefficient [m2m2/kg]
 - `x_cr_area_over_mass`: solar radiation coefficient . A/m (ballistic coefficient equivalent)
 - `x_h_apo`: apogee (-RearthRearth) [km]
 - `x_h_per`: perigee (-RearthRearth)[km]
 - `x_j2k_ecc`: eccentricity
 - `x_j2k_inc`: inclination [deg]
 - `x_j2k_sma`: semi-major axis [km]
 - `x_sedr`: energy dissipation rate [W/kg]
 - `x_span`: size used by the collision risk computation algorithm (minimum 2 m diameter assumed for the chaser) [m]
 - `x_rcs_estimate`: radar cross-sectional area [m2m2]
 - `x_actual_od_span`: actual length of update interval for orbit determination [days]
 - `x_obs_available`: number of observations available for orbit determination (per CDM)
 - `x_obs_used`: number of observations used for orbit determination (per CDM)
 - `x_recommended_od_span`: recommended length of update interval for orbit determination [days]
 - `x_residuals_accepted`: orbit determination residuals
 - `x_time_lastob_end`: end of the time interval in days (with respect to the CDM creation epoch) of the last accepted observation used in the orbit determination
 - `x_time_lastob_start`: start of the time in days (with respect to the CDM creation epoch) of the last accepted observation used in the orbit determination
 - `x_weighted_rms`: root-mean-square in least-squares orbit determination

# Exploratory Data Analysis (EDA)

The development of any Machine Learning model requires a solid knowledge of the data available for its training and subsequent development in order to guarantee a successful deployment and applicability to real-world problems. For this purpose, in this section we develop a comprehensive *Exploratory Data Analysis* (EDA) on the Kelvins Collision Avoidance Challenge `training` and `test` with two main objectives in mind:
 - Acquire a solid understanding of real data received in a conventional Conjunction Data Message (CDM); this involves a basic analysis of the data distribution and its potential common patterns for ***Target*** and ***Chaser*** objects, and cluster identification for conjunction events.
 
- Develop a Synthetic Data Generation (SDG) process that can reliably produce additional virtual (non-existing) data with the objective to enrich the Time-Series Forecasting Deep Learning model and improve its performance in production by reinforcing its training process.

For this purpose, this notebook is structured in the following sections:

1. Data import and initial exploration.
2. Data distribution analysis.
3. Probability Density estimation.

In [ ]:
# Import libraries required for EDA
import pandas as pd
import numpy as np
import math
import scipy.stats as st
import os
import warnings
import requests
import time
import json

# Import Scikit-learn required libraries
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Import function to clear output
from IPython.display import clear_output

# Import file system libraries
from pathlib import Path

# Import matplotlib library and setup environment for plots
%matplotlib inline
%config InlineBackend.figure_format='retina'
from matplotlib import rc, gridspec
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors 

# Set rendering parameters to use TeX font.
if not '/private/var/' in os.getcwd():
    rc('font', **{'family': 'serif', 'serif': ['Computer Modern'], 'size': 11})
    rc('text', usetex=True)

## 1. - Data import and initial exploration

In this section, we import and explore the data provided in the competition with the aim to identify all those fields relevant for the synthetic data generation process and the Time-Series Forecasting problem.

In [ ]:
# Get current working directory path for the tool parent folder and print it.
parent_folder = 'Tool'
cwd = str(Path(os.getcwd()[:os.getcwd().index(parent_folder)+len(parent_folder)]))
print('Parent working directory: \n%s' % cwd)

# Set maximum number of columns to show as None
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# Import training dataset
df = pd.read_csv(os.path.join(cwd,'data','esa-challenge','train_data.csv'), 
                 sep=',', header=0, index_col=None, skipinitialspace=False)

# Sort values of dataframe by event_id and time_to_tca and re-index
df.sort_values(by=['event_id', 'time_to_tca'], axis='index', 
               ascending=[True,False], inplace=True, ignore_index=True)

# Show first n rows of dataframe
df.head(10)


As described in the introduction of Kelvins competition, both `training` and `test` datasets contain several CDMs per `event_id`, being the **last CDM** for a given `event_id` the one with the **most accurate collision probability risk estimation**. Following the same assumption for the keplerian elements measurements, a subset `df_lastCDM` from the origin datasets is created containing only the last CDM. 

This subset will be used in subsequent sections of the EDA to understand how population of ASOs is distributed across its different dimensions so that synthetic data can be generated from that point. 

In [ ]:
# Get only last CDM data from every event_id
df_lastCDM = df.drop_duplicates('event_id', keep='last')

# Show first n rows of dataframe with only final CDMs.
df_lastCDM.head(10)

## 2. - Data distribution analysis

In order to generate synthetic data that reliably describe the current problem of ASOs population in the near Earth environment it is crucial to analyse their keplerian orbital elements. This analysis must be performed on the two types of objects involved in the conjunction events (*Target* and *Chaser*), and the orbital elements data to be considered the following:

 - `x_h_apo` (Apogee altitude)
 - `x_h_per` (Perigee)
 - `x_j2k_ecc` (Eccentricity)
 - `x_j2k_inc` (Inclination)
 - `x_j2k_sma` (Semi-major axis)

In addition to the orbital elements, the analysis of the `miss_distance` data distribution is also of crucial interest as it constitutes one of the starting points in the synthetic data generation process.


In [ ]:
# Define function to define upper and lower limits for standard data in a population
def outliers_boundaries(data, threshold = 2.0):

    Q1 = np.quantile(data,0.25)
    Q3 = np.quantile(data,0.75)
    IQR = st.iqr(data)
    
    return (max(0, (Q1-IQR*threshold)), (Q3+IQR*threshold))

In [ ]:
# Define function to estimate adequate number of bins for histograms.
def nbins(data, rule):
    
    # Get the number of items within the dataset
    n = len(data)
    
    # Compute the histogram bins size (range)
    bins_size = {'sturge': 1 + 3.322*np.log(n),    
                 'scott': 3.49*np.std(data)*n**(-1/3),                       
                 'rice': 2*n**(1/3),                         
                 'fd': 2*st.iqr(data)*n**(-1/3)}
    
    # Return number of bins
    return math.ceil((data.max() - data.min())/bins_size[rule])

### 2.1. - Distance at closest approach distribution analysis (`miss_distance`)

The starting point for the process to generate additional synthetic data to be used to improve the training of the DL model is the analysis of the `miss_distance`. By analysing the distribution of the miss distance it is possible increase the frequency of conjunctions throughout time while ensuring reliability of data. To do so, we first have a look to the data distribution using an histogram plot as follows:

In [ ]:
# Get data to plot
data = df_lastCDM['miss_distance'].to_numpy()

# Plot histogram of miss_distance
plt.figure(figsize=(7,3))
plt.hist(data, bins = nbins(data, 'fd'), edgecolor='white')
plt.xlim(0, data.max())
plt.ylabel(r'Nb. of events')
plt.xlabel(r'Miss distance (m)')
plt.grid(True, linestyle='--')
plt.show()

### 2.2. - Keplerian elements distribution analysis

In this section the following orbital elements are analysed:

In [ ]:
# Define orbital elements columns dictionary
keColumns = {'_h_per':   'Perigee (km)',
             '_j2k_sma': 'Semi-major axis (km)',
             '_j2k_inc': 'Inclination (deg)',
             '_j2k_ecc': 'Eccentricity', 
             }

# Iterate over all orbital elements
for eType, eName in keColumns.items():
    
    # Get orbital element data from both target and chaser objects to compute outliers boundaries
    data = pd.concat([df_lastCDM['t' + eType], df_lastCDM['c' + eType]]).to_numpy()
    
    # Calculate number of outliers for the orbital element
    std_lims = outliers_boundaries(data, 2.0)
    outliers = (data<std_lims[0]) | (data>std_lims[1])

    
    # Create figure object
    fig, axs = plt.subplots(nrows=2, figsize=(10, 5), squeeze=True, 
                            sharex=True, sharey=True)
    
    # Set plot title to indicate boundaries for standard data and outliers excluded from histogram.
    axs[0].set_title(r'Std. data limits: [{:.3f}, {:.3f}] \qquad Objects: {:d} \qquad Outliers: {:d} ({:.2f}\%)'\
                     .format(std_lims[0], std_lims[1], len(data),  
                             outliers.sum(), outliers.sum()/len(df_lastCDM.index)*100), 
                     fontsize=10)
    
    # Get minimum and maximum from data
    xlim = (std_lims[0], std_lims[1])
    
    # Plot charts in subplot for both objects
    colors = ['tab:blue','tab:orange']
    
    # Calculate number of bins to plot histogram 
    bins = nbins(data[outliers==False], 'fd')
    
    for i, obj in enumerate(['target', 'chaser']):
        
        
        # Plot histogram for the object and keplerian element.
        axs[i].hist(df_lastCDM[obj[0] + eType], bins = bins, 
                    color=colors[i], edgecolor='white', range=xlim, 
                    label=r'' + obj.capitalize())
        
        # Get the limits of Y-axis to reformat plot.
        ylim = axs[i].get_ylim()
        
        yscale = 10**(math.floor(math.log(ylim[1], 10)))
        
        ylim = (ylim[0], math.ceil(ylim[1]/yscale)*yscale)

        axs[i].grid(True, linestyle='--')
        axs[i].set_yticks(np.linspace(ylim[0],ylim[1],5))
        axs[i].set_xlim(xlim)
        axs[i].set_ylim(ylim)
        
        axs[i].set_ylabel(r'Nb. of objects')
        
        # Plot label to identify which object the chart belongs to
        axs[i].text(0.95, 0.95, r'' + obj.capitalize(), size=10, ha='center', va='top', 
                    c=colors[i], transform=axs[i].transAxes, 
                    bbox=dict(facecolor='white', alpha=0.75, edgecolor='white', pad=-1))

    
    axs[1].set_xlabel(r'' + eName)
    
    plt.show()


# 3. - Probability Density Estimation for synthetic data generation

[Introduction to Probability Density Estimation](https://machinelearningmastery.com/probability-density-estimation/)

This analysis includes the two different ways to estimate the probability density associated to a given data distribution:

 - **Parametric**: the statistical distribution of the real data is described by an analytical and well-known statistical probability distribution by estimating the parameters implicit in the function.
 
 - **Non-parametric**: alternative method when the data distribution cannot be appropriately described analytically by a common probability distribution or cannot be easily made to fit the distribution (as it happens with multi-modal distributions). Among the multiple non-parametric methods available for Probability Density Estimation, the method used for estimating the probability density function of a continuous random variable in this analysis is the so-called Kernel Density Estimation (KDE).
 
The probability density function with wich all relevant continuous variables for which new data shall be generated, are fitted using both parametric and non-parametric methods and compared between each other to identify the one with the lowest Error Sum of Squares (SSE). For the parametric approach, a function is defined to find the statistical distribution that best fits the real data distribution of all existing continuous distributions available in [SciPy.org](https://docs.scipy.org/doc/scipy/reference/stats.html). In the non-parametric approach the `KernelDensity` estimator is used from [Scikit-learn.org](https://scikit-learn.org/stable/modules/density.html) to fit a function to the continuous variable.

### 3.1. - Parametric Probability Density estimation

The process to fit a parametric probability density estimation follows a series of steps for which it is convenient to develop segregated functions. For this purpose, the following functions are defined:
 - `fit_distribution`: Fits a statistical continuous distribution to the input data. It returns the distribution, its parameters adjusted, and the resulting SSE.
 - `make_pdf`: It creates an array to represent the probability density function using a given fitted distribution and its parameters.
 - `plot_dist`: Plots the probability density function and the data distribution histogram on a single chart for a visual interpretation of the probability density estimation. It supports the reader/data analyst to check appropriate fitting; some distributions may return low SSE but does not appropriately describe de distribution.


In [ ]:
# Define function to fit statistical distribution to the data.
def fit_distribution(distribution, data, bins):
    
    # Get histogram of original data
    y, x = np.histogram(data, bins=bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0
    
    # Try to fit the distribution
    tic = time.process_time()
    try:
        # Ignore warnings from data that can"t be fit
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")

            # Fit dist to data
            fitting_output = distribution.fit(data)

            # Separate parts of parameters
            parameters = {'loc':        fitting_output[-2],
                          'scale':      fitting_output[-1],
                          'arg':        fitting_output[:-2],
                          'arg_names':  distribution.shapes.split(", ") if distribution.shapes!=None else []}
    
            # Calculate fitted PDF and error with fit in distribution
            pdf = distribution.pdf(x, loc=parameters['loc'], scale=parameters['scale'], *parameters['arg'])
            sse = np.sum(np.power(y - pdf, 2.0))

    except Exception:
        
        parameters, sse = None, np.inf
        pass
    
    toc = time.process_time()
    
    # identify if this distribution is better
    output = {'distribution': distribution, 'name': distribution.name, 
              'parameters': parameters, 'sse': sse}
    
    output['performance'] = {'processing_time': toc-tic, 'bins':bins}
    
    return output

In [ ]:
# Define function to generate distributions's Probability Distribution Function 
def make_pdf(distribution, parameters, size=10000):

    # Separate parts of parameters
    arg     = parameters['arg']
    loc     = parameters['loc']
    scale   = parameters['scale']

    # Get same start and end points of distribution
    start = distribution.ppf(1e-4, *arg, loc=loc, scale=scale) \
        if arg else distribution.ppf(1e-4, loc=loc, scale=scale)
    end = distribution.ppf(1-1e-4, *arg, loc=loc, scale=scale) \
        if arg else distribution.ppf(1-1e-4, loc=loc, scale=scale)

    # Build PDF and turn into pandas Series
    x = np.linspace(start, end, size)
    y = distribution.pdf(x, loc=loc, scale=scale, *arg)
    pdf = pd.Series(y, x)

    return pdf

In [ ]:
## Define function to plot Probability Density Function with the histogram on the same chart
def plot_distribution(data, distribution, parameters, bins):
    
    # Make Probability Density Function with distribution parameters 
    pdf = make_pdf(distribution, parameters)
            
    # Create string with all parameters to print into the plot's title
    title = r'Stats model: \texttt{' + distribution.name + r'}\small' + '\n' + \
                "\quad ".join(["{} = {:0.3f}".format(k,v) \
                for k,v in zip(parameters['arg_names'] + ["loc", "scale"], 
                               parameters['arg'] + parameters['loc'] + parameters['scale'])])
    

    # Display plot
    plt.figure(figsize=(7,3))
    
    ax = plt.axes()
    t = ax.text(0.5, 0.975, title, size=10, ha='center', va='top', \
                c='black', transform=ax.transAxes, \
                bbox=dict(facecolor='white', alpha=0.75, edgecolor='white', pad=-1))
    
    plt.plot(pdf.index.to_numpy(), pdf.values, lw=1.5, color = "orange", label="PDF")
    n, xbins, patches = plt.hist(data, bins=bins, density=True, histtype='bar', 
                                 color="dimgrey", edgecolor = "white", label="Data")
    
    bins_width = (xbins[1]-xbins[0])
    plt.xlim(xbins[0] - bins_width, xbins[-1] + bins_width)
    plt.ylim(0, np.max(n)*1.5)
    
    plt.title(r'Actual data distribution vs. fitted PDF', fontsize=12)
    # plt.xlabel(data.name)
    plt.ylabel(r"Probability density")
    plt.grid(True, linestyle="dashed", alpha=0.5)
    plt.legend(loc="best", fontsize=10)
    plt.show()
    
    return

#### Finding the best statistical model to recreate real data distributions.

As described previously, for the parametric approach the function `find_best_distribution` is defined to find the statistical distribution that best fits the real data distribution of all existing continuous distributions available in [SciPy.org](https://docs.scipy.org/doc/scipy/reference/stats.html). To do so, the algorithm retrieves all existing continuous distributions, tries to fit it to the real data, computes the SSE and compare to the best fit found so far. It returns the best statistical distribution which corresponds to the one with the lowest SSE.

In [ ]:
# Define function to get the list of continuous distributions from SciPy.org website
def get_scipy_distributions():
    try:
        # Get Continuous distributions table from SciPy website.
        url = 'https://docs.scipy.org/doc/scipy/reference/stats.html'
        tbody = pd.read_html(requests.get(url).content)[1]
        
        # Create pandas dataframe and save CSV local file to be able to use the notebook offline.
        df_distributions = pd.DataFrame(data = tbody[0].to_list(), columns=['scipy_distributions'])
        df_distributions.to_csv(os.path.join(cwd,'temp_scipy-distributions.csv'), sep=',')
        
    except Exception:
        print("Could not read SciPy.org website. Importing data from local file...")
        # Import training dataset
        df_distributions = pd.read_csv(os.path.join(cwd,'data','notebook-outputs','temp_scipy-distributions.csv'), 
                            sep=',', header=0, index_col=None, skipinitialspace=False)

        pass
    
    # Evaluate list of objects in str format to convert it into a Python list
    distributions_list = []
    
    # Iterate through all the continous distributions on the website and evaluate it
    # to discard those that are not compatible with the library version installed.
    for distribution_i in df_distributions['scipy_distributions'].to_list():
        try:
            distributions_list.append(eval('st.' + distribution_i))
            print(distribution_i)
        except Exception:
            pass

    return distributions_list

In [ ]:
# Define function to find best fitted distribution (according to lowest SSE)
def find_best_distribution(data, bins, exclusion_list=[]):
    
    # Evaluate list of objects in str format to convert it into a Python list
    distributions_list = get_scipy_distributions()
    
    # Initialize best holder using the norm distribution
    best_holder = fit_distribution(st.norm, data, bins)
    
    fitting_results = []

    # Estimate distribution parameters from data
    for i, distribution_i in enumerate(distributions_list):
        
        # If statistical distribution is in the exclusion list skip fitting process.
        if distribution_i.name in exclusion_list: continue
        
        # Fit stdist to real data
        fitting_output = fit_distribution(distribution_i, data, bins)

        clear_output(wait=True)

        print('Progress %5.1f%% (%3d/%3d)  Best: %10s (SSE: %1.3e)  Distribution: %10s (SSE: %1.3e) \t' %
              ((i+1)/len(distributions_list)*100, i, len(distributions_list)-1, 
               best_holder['name'], best_holder['sse'], 
               fitting_output['name'], fitting_output['sse']), end='\r')
        
        # If it improves the current best distribution, reassign best distribution
        if best_holder['sse'] > fitting_output['sse'] > 0: best_holder = fitting_output
        
        fitting_results.append([fitting_output['name'], fitting_output['parameters'], 
                                fitting_output['sse'], fitting_output['performance']])
        
    # Sort values of dataframe by sse ascending and and re-index.
    ranking = pd.DataFrame(data=fitting_results, columns=['distribution', 'parameters', 'sse', 'performance'])
    ranking.sort_values(by=['sse'], axis='index', ascending=[True], 
                              inplace=True, ignore_index=True)
    
    # Clear output to print final results
    clear_output(wait=True)
    
    # Print final results
    # print("\nBest distribution: \n %s" % (json.dumps(best_holder.pop('distribution'), sort_keys=True, indent=4)))
    
    return best_holder, ranking

#### Evaluation of  `miss_distance` using the parametric approach

Computing `find_best_distribution` function over all relevant continuous variables the results are the following:

In [ ]:
# Set column name to study and remove outliers.
data = df_lastCDM['miss_distance']

# Find distribution that best fits the data
# distribution = fit_distribution(st.kappa4, data, bins = nbins(data, 'fd'))
distribution, ranking = find_best_distribution(data, bins = nbins(data, 'fd'), 
                                               exclusion_list = ['studentized_range', 
                                                                 'levy_l_gen', 
                                                                 'levy_stable'])
    
# Print plot including histogram and distribution fitted
plot_distribution(data, distribution['distribution'], distribution['parameters'], bins = nbins(data, 'fd'))
# ranking.head(10)

The continuous statistical model that best describes the `miss_distance` data distribution is `kappa4`. In order to compare side by side real data versus the synthetic data a histogram with the same number of events is generated using the trained model below: 

In [ ]:
# Get best results parameters from the ranking dataset on miss_distance variable
best_stmodel = ranking.loc[0].to_dict()
# print(json.dumps(best_stmodel, sort_keys=True, indent=4))

# Get the distribution object from the best result
distribution = getattr(st, best_stmodel['distribution'])

# Generate random distribution using rvs function.
synthetic_data = distribution.rvs(*best_stmodel['parameters']['arg'],
                                 loc   = best_stmodel['parameters']['loc'],
                                 scale = best_stmodel['parameters']['scale'], 
                                 size=500000, random_state=1)

# Plot actual vs synthetic data for comparison
plt.figure(figsize=(7, 3))
plt.hist([data, synthetic_data], density=True,
         bins = nbins(data, 'fd'), 
         label=['Actual', 'Synthetic'], 
         color=['dimgrey','lightskyblue'])
plt.ylabel(r'Probability density of ocurrence', fontsize=10)
plt.xlabel(r'Miss distance (m)', fontsize=10)
plt.title(r'Actual vs synthetic data distribution'+ '\n' +'\small (Stats model ' + distribution.name + ')')
plt.grid(True, linestyle='dashed', alpha=0.5)
plt.legend(loc='best', fontsize=10)

plt.show()

### 3.2. - Non-Parametric Probability Density Estimation using KDE

The keplerian elements that describle the orbits for both targets and chasers objects shows multimodal distributions that cannot be described by continuous stats models. As a consequence, the Non-parametric Probability Density Estimation (PDE) with KDE shall be used in order to fit a model that allows for synthetic data generation with a realistic distribution.

In [ ]:
colors = list(mcolors.TABLEAU_COLORS.values())

# Run through both type of objects
for keColumn, keLabel in keColumns.items():

    # Get the data upon which the clustering shall be done
    data = np.array(df_lastCDM['t' + keColumn]).reshape(-1, 1)

    # Get number of bins used in a global histogram for better readability
    bins = nbins(data, 'fd')

    # Use Silhouette method to compute optimal number of clusters

    potential_clusters = np.arange(2, 11,1)
    silhouette_avg = []
    for n_clusters in potential_clusters:

        # Initialise the clusterer object
        clusterer = KMeans(n_clusters=n_clusters, random_state=0).fit(data)

        # Calculate silhouette score and append it to the array
        silhouette_avg.append(silhouette_score(data, clusterer.labels_))

    # Get maximum Silhouette score that indicate optimal number of clusters
    max_silhouette_score = np.max(silhouette_avg)
    n_clusters = np.min(potential_clusters[silhouette_avg==max_silhouette_score])

    # Use KMeans algorithm to identify all different clusters in 1D
    clusterer = KMeans(n_clusters=n_clusters, random_state=0).fit(data)
    
    # Re-label clusters to be sorted by orbit region from lower to upper
    # in order to visually identify cluster labels from plots
    clusters_centers = clusterer.cluster_centers_.flatten()
    clusters_labels = clusterer.predict(clusters_centers.reshape(-1,1))
    
    labels  = clusterer.labels_.flatten()

    custom_labels = np.zeros_like(labels)
    for c, center in enumerate(np.sort(clusters_centers)):
        label = clusters_labels[clusters_centers==center]
        custom_labels[labels == label] = c


    # Create the figure object
    fig = plt.figure(figsize=(10,2.5))

    # Create grid for different subplots
    spec = gridspec.GridSpec(ncols=2, nrows=1, width_ratios=[2, 3], wspace=0.25)

    # Plot Silhouette chart
    ax0 = fig.add_subplot(spec[0])
    ax0.plot(potential_clusters, silhouette_avg,'kx--')
    ax0.plot(n_clusters, max_silhouette_score, 'ro')
    ax0.set_ylabel(r'Silhouette score')
    ax0.set_xlabel(r'Nb. of clusters')
    ax0.set_ylim(0.5,1)
    ax0.set_xticks(potential_clusters)

    # Plot histogram grouped by clusters
    ax1 = fig.add_subplot(spec[1])
    N, bins, patches = ax1.hist(data, bins = bins, edgecolor = 'white')
    ax1.set_ylabel(r'Nb. of events')
    ax1.set_xlabel(r'' + keLabel)
    
    bins_per_cluster = dict(zip(clusters_labels, [0]*len(clusters_labels)))
    print(bins_per_cluster)

    for i in range(len(patches)):

        # Get the cluster to which the bin belongs to
        bin_cluster = custom_labels[(data<=bins[i+1]).flatten() & (data>bins[i]).flatten()]
        
        if len(bin_cluster)>0: 
            bins_per_cluster[bin_cluster[0]] += 1
            # Change histogram bar color if it belongs to a cluster
            patches[i].set_facecolor(colors[np.max(bin_cluster)])

    ax0.grid(True, linestyle="dashed", alpha=0.5)
    ax1.grid(True, linestyle="dashed", alpha=0.5)
    
    fig.suptitle(r'Data distribution cluster analysis for \texttt{t' + keColumn + '}' + '\n'
                 '\small (Optimum number of clusters = ' + str(n_clusters) + ')', 
                 fontsize=12, y=1.05)
    
    plt.show()
    print(bins_per_cluster)
    
    break

In [ ]:
# Get the data upon which the clustering shall be done
data = np.array(df_lastCDM['t_h_per']).reshape(-1, 1)

# Clusters 1, 3 can be fitted with norm (cluster 4 could potentially be included) 
# Cluster 2 can be fitted with mielke
n_cluster = 4
sub_data = data[custom_labels==n_cluster]
bins = bins_per_cluster[n_cluster]

# Find distribution that best fits the data
# distribution = fit_distribution(st.kappa4, data, bins = nbins(data, 'fd'))
distribution, ranking = find_best_distribution(sub_data, bins = bins, 
                                               exclusion_list = ['studentized_range', 
                                                                 'levy_l_gen', 
                                                                 'levy_stable'])
    
# Print plot including histogram and distribution fitted
plot_distribution(sub_data, distribution['distribution'], distribution['parameters'], bins=bins)


distribution = fit_distribution(st.norm, sub_data, bins = bins)
plot_distribution(sub_data, distribution['distribution'], distribution['parameters'], bins=bins)    


ranking.head(20)


#### Probability Density Estimation using Scikit-Learn

In [ ]:
# Initialize array with all available kernels in KernelDensity
kernels = ["gaussian", "tophat", "epanechnikov", "exponential", "linear", "cosine"]

# Initialize array with different values for the bandwidth
bandwidths = np.arange(2,20,2)


# Get the data upon which the clustering shall be done
data = np.array(df_lastCDM['t_h_per']).reshape(-1, 1)

# Clusters 1, 3 can be fitted with norm 
# Cluster 2 can be fitted with mielke
n_cluster = 2
sub_data = data[custom_labels==n_cluster]
bins = bins_per_cluster[n_cluster]

# Get probability density from real data
dens_data, bin_edges = np.histogram(sub_data, bins=bins, density=True)

X = (bin_edges + np.roll(bin_edges, -1))[:-1] / 2.0


# Create plot object to compare all kernels as a function of the bandwidth
plt.figure(figsize=(6,3))

# Iterate through all kernels
for kernel in kernels:
    
    # Initialize array to store SSE
    sse = np.zeros_like(bandwidths, dtype=float)

    # Iterate through bandwidths
    for i, bandwidth in enumerate(bandwidths):


        # Fit the Kernel Density Estimator
        kde = KernelDensity(kernel=kernel, bandwidth=bandwidth).fit(sub_data)

        # Compute the exponential of the log-likelihood of each sample under the model.
        dens_estimated = np.exp(kde.score_samples(X.reshape(-1,1)))

        # Compute Sum of Squared Errors (SSE) for the density estimation
        sse[i] = np.sum((dens_estimated - dens_data)**2)

    print("Min. SSE: %.4e\t Bandwidth: %.3f\t Kernel: %s" % (np.min(sse), bandwidths[sse==np.min(sse)][0], kernel))

    plt.plot(bandwidths, sse, label=r'' + kernel)

plt.title(r'Comparison between Kernels given a bandwidth')

plt.ylabel(r'Sum of Squared Errors (SSE)')
plt.xlabel(r'Bandwidth')

# Define X-axis ticks and limits
plt.xticks(np.arange(0.0,bandwidths.max(),0.5))
plt.xlim(0.0, bandwidths.max())

# Define Y-axis ticks and limits
#plt.yticks(np.linspace(0.0, sse.max(),10))
#plt.ylim(0.0, sse.max())

plt.grid(True, linestyle='--')
plt.legend(loc='best')
plt.show()


for kernel in kernels:

    # Create plot object to compare all kernels as a function of the bandwidth
    plt.figure(figsize=(6,3))

    
    plt.hist(sub_data, label=r'Real data', density=True)
    for bandwidth in bandwidths:
        
    
        # Fit the Kernel Density Estimator
        kde = KernelDensity(kernel=kernel, bandwidth=bandwidth).fit(sub_data)

        # Compute the exponential of the log-likelihood of each sample under the model.
        X_estimated = np.linspace(np.min(X), np.max(X), 100)
        dens_estimated = np.exp(kde.score_samples(X_estimated.reshape(-1,1)))
        
        plt.plot(X_estimated, dens_estimated, label=r'' +'{:.2f}'.format(bandwidth))
    plt.title(r'' + kernel)
    plt.ylabel(r'Probability Density')
    plt.xlabel(r'Bandwidth')

    plt.grid(True, linestyle='--')
    plt.legend(loc='best')
    plt.show()
    


In [ ]:
data = df_lastCDM['miss_distance'].to_numpy().reshape(-1,1)


grid = GridSearchCV(KernelDensity(kernel='exponential'),
                    {'bandwidth': np.linspace(120, 130, 20)},
                    cv=5) # 20-fold cross-validation
grid.fit(data)
print(grid.best_params_)

In [ ]:
# Get both object's data
data = sub_data

# Create kernel to estimate probability over the population
kde = st.gaussian_kde(data, bw_method='scott')
    
# Get minimum and maximum from data
x = np.linspace(data.min(), data.max(), 100)

# Plot the probability density distribution estimated function
plt.figure(figsize=(6,3), layout='tight')
plt.plot(x, kde(x))
plt.title(r'Probability density estimate')
plt.xlabel(r'Miss distance (km)')
plt.grid(True, linestyle='--')
plt.show()

In [ ]:
# Run through both type of objects
for eType, eName in keColumns.items():
    
    # Get both object's data
    data = df_lastCDM[['t' + eType,'c' + eType]]
    
    # Get minimum and maximum from data
    xlim = (data.min().min(), data.max().max())
    
    x = np.linspace(xlim[0], xlim[1], 100)
    
    plt.figure(figsize=(6,3), layout='tight')
    
    for oType, oPreffix in objects.items():
    
        # Create kernel to estimate probability over the population
        kde = st.gaussian_kde(data[oPreffix + eType], bw_method='scott')
        
        plt.plot(x, kde(x), label=r'' + oType.capitalize())
        
    # print('KDE Bandwith method = %.4f' % kde.n**(-1./(kde.d+4)))
    plt.title(r'Probability density estimate')
    plt.xlabel(eName)
    plt.grid(True, linestyle='--')
    plt.legend(loc='best')
    plt.show()